# Automatic Configuration

In [1]:
from jupyter_ai_agent_magics import config
from jupyter_ai_agent_magics.config import ModelProvider
from jupyter_ai_agent_magics.config import ModelProviderSettings
config_manager = config.ConfigManager.instance

import ipywidgets as widgets
from IPython.display import display
from typing import Dict, List, Optional
from enum import StrEnum
from dataclasses import dataclass


class MultiProviderModelSelector:
    """Widget for selecting AI model providers and their specific configurations."""
    
    def __init__(self, config_manager=None):
        self.config_manager = config_manager
        
        # Define available models for each provider
        self.provider_models = {
            # https://docs.anthropic.com/en/docs/about-claude/models/overview
            ModelProvider.Anthropic: {
                "Claude Sonnet 4": "claude-sonnet-4-20250514",
                "Claude Opus 4.1": "claude-opus-4-1-20250805",
                "Claude Opus 4": "claude-opus-4-20250514",
                "Claude 3.5 Sonnet": "claude-3-5-sonnet-20241022",
                "Claude 3.5 Haiku": "claude-3-5-haiku-20241022", 
                "Claude 3 Opus": "claude-3-opus-20240229",
                "Claude 3 Sonnet": "claude-3-sonnet-20240229",
                "Claude 3 Haiku": "claude-3-haiku-20240307"
            },
            # https://cloud.google.com/vertex-ai/generative-ai/docs/partner-models/claude
            # https://docs.anthropic.com/en/docs/about-claude/models/overview
            ModelProvider.AnthropicVertex: {
                "Claude Sonnet 4": "claude-sonnet-4@20250514",
                "Claude Opus 4": "claude-opus-4@20250514",
                "Claude 3.7 Sonnet": "claude-3-7-sonnet@20250219",
                "Claude 3.5 Sonnet v2": "claude-3-5-sonnet-v2@20241022",
                "Claude 3.5 Haiku": "claude-3-5-haiku@20241022",
                "Claude 3.5 Sonnet": "claude-3-5-sonnet@20240620",
                "Claude 3 Opus": "claude-3-opus@20240229",
                "Claude 3 Haiku": "claude-3-haiku@20240307",
                "Claude 3 Sonnet": "claude-3-sonnet@20240229"
            },
            #https://ai.google.dev/gemini-api/docs/models
            ModelProvider.Google: {
                "Gemini 2.5 Pro": "gemini-2.5-pro",
                "Gemini 2.5 Flash": "gemini-2.5-flash",
                "Gemini 2.0 Pro": "gemini-2.0-pro",
                "Gemini 2.0 Flash": "gemini-2.0-flash",
                "Gemini 1.5 Pro": "gemini-1.5-pro",
                "Gemini 1.5 Flash": "gemini-1.5-flash",
            },
            # https://platform.openai.com/docs/pricing
            ModelProvider.OpenAI: {
                "GPT-5": "gpt-5",
                "GPT-5-mini": "gpt-5-mini",
                "GPT-5-nano": "gpt-5-nano",
                "GPT-4.1": "gpt-4.1",
                "GPT-4.1 Nano": "gpt-4.1-nano",
                "GPT-4.1 Mini": "gpt-4.1-mini",
                "GPT-4o": "gpt-4o",
                "GPT-4o Mini": "gpt-4o-mini",
                "GPT-4 Turbo": "gpt-4-turbo",
                "GPT-4": "gpt-4",
                "GPT-3.5 Turbo": "gpt-3.5-turbo"
            }
        }
        
        # Vertex AI regions
        self.vertex_regions = {
            "US Central 1": "us-central1",
            "US East 1": "us-east1",
            "US East 4": "us-east4", 
            "US West 1": "us-west1",
            "US West 4": "us-west4",
            "Europe West 1": "europe-west1",
            "Europe West 3": "europe-west3",
            "Europe West 4": "europe-west4",
            "Asia Northeast 1": "asia-northeast1",
            "Asia Southeast 1": "asia-southeast1"
        }
        
        # Google Cloud regions (for Google provider)
        self.google_regions = {
            "US Central 1": "us-central1",
            "US East 1": "us-east1",
            "Europe West 1": "europe-west1",
            "Asia Northeast 1": "asia-northeast1"
        }
        
        # Anthropic Vertex model-specific region mappings
        self.vertex_model_regions = {
            "claude-sonnet-4@20250514": {
                "US East 5": "us-east5",
                "Europe West 4": "europe-west4", 
                "Global": "GLOBAL"
            },
            "claude-opus-4@20250514": {
                "US East 5": "us-east5",
                "Europe West 4": "europe-west4"
            },
            "claude-3-7-sonnet@20250219": {
                "US East 5": "us-east5",
                "Europe West 1": "europe-west1",
                "Europe West 4": "europe-west4",
                "Global": "GLOBAL"
            },
            "claude-3-5-sonnet-v2@20241022": {
                "US East 5": "us-east5",
                "Europe West 1": "europe-west1",
                "Global": "GLOBAL"
            },
            "claude-3-5-haiku@20241022": {
                "US East 5": "us-east5"
            },
            "claude-3-5-sonnet@20240620": {
                "US East 5": "us-east5",
                "Europe West 1": "europe-west1",
                "Asia Southeast 1": "asia-southeast1"
            },
            "claude-3-opus@20240229": {
                "US East 5": "us-east5"
            },
            "claude-3-haiku@20240307": {
                "US East 5": "us-east5",
                "Europe West 1": "europe-west1",
                "Asia Southeast 1": "asia-southeast1"
            },
            "claude-3-sonnet@20240229": {
                "US East 5": "us-east5"
            }
        }
        
        self.setup_widgets()
        self.load_from_config()
        
    def setup_widgets(self):
        """Create and configure all widgets."""
        
        # Provider selector
        self.provider_dropdown = widgets.Dropdown(
            options=[(provider.value.replace('_', ' ').title(), provider) 
                    for provider in ModelProvider],
            value=ModelProvider.Anthropic,
            description="Provider:",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Model selector (will be updated based on provider)
        self.model_dropdown = widgets.Dropdown(
            options=[],
            description="Model:",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        # Region selector (for Vertex/Google providers)
        self.region_dropdown = widgets.Dropdown(
            options=[],
            description="Region:",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        
        # Project ID textbox (for Vertex/Google providers)
        self.project_textbox = widgets.Text(
            description="Project ID:",
            placeholder="Enter your GCP project ID",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        # API Key textbox
        self.api_key_textbox = widgets.Text(
            description="API Key:",
            placeholder="Enter API key (optional if using env vars)",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        # Temperature slider
        self.temperature_slider = widgets.FloatSlider(
            value=0.7,
            min=0.0,
            max=2.0,
            step=0.1,
            description="Temperature:",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        # Enable/disable temperature
        self.use_temperature_checkbox = widgets.Checkbox(
            value=True,
            description="Use custom temperature",
            style={'description_width': 'initial'}
        )
        
        # Output widget
        self.output = widgets.Output()
        
        # Save button
        self.save_button = widgets.Button(
            description="Save Configuration",
            button_style='primary',
            layout=widgets.Layout(width='200px')
        )
        
        # Container for conditional widgets (region, project)
        self.conditional_container = widgets.VBox([])
        
        # Main container
        self.container = widgets.VBox([
            widgets.HTML("<h3>AI Model Provider Configuration</h3>"),
            self.provider_dropdown,
            self.model_dropdown,
            self.conditional_container,  # Will contain region/project widgets
            widgets.HBox([self.use_temperature_checkbox, self.temperature_slider]),
            self.save_button,
            widgets.HTML("<h4>Configuration Code:</h4>"),
            self.output
        ])
        
        # Set up observers
        self.provider_dropdown.observe(self.on_provider_change, names='value')
        self.model_dropdown.observe(self.on_model_change, names='value')
        self.region_dropdown.observe(self.update_output, names='value')
        self.project_textbox.observe(self.update_output, names='value')
        self.api_key_textbox.observe(self.update_output, names='value')
        self.temperature_slider.observe(self.update_output, names='value')
        self.use_temperature_checkbox.observe(self.on_temperature_toggle, names='value')
        self.save_button.on_click(self.save_to_config)
        
        # Initialize widgets
        self.on_provider_change({'new': self.provider_dropdown.value})
        
    def on_provider_change(self, change):
        """Handle provider selection change."""
        provider = change['new']
        
        # Update model options
        models = self.provider_models[provider]
        self.model_dropdown.options = [(name, code) for name, code in models.items()]
        self.model_dropdown.value = list(models.values())[0]  # Select first model
        
        # Update conditional widgets based on provider
        conditional_widgets = []
        
        if provider == ModelProvider.AnthropicVertex:
            # Show region and project for Vertex
            # Use model-specific regions if available, otherwise default to general vertex regions
            model_code = list(models.values())[0]  # Get first model code
            if model_code in self.vertex_model_regions:
                regions = self.vertex_model_regions[model_code]
                self.region_dropdown.options = [(name, code) for name, code in regions.items()]
                self.region_dropdown.value = list(regions.values())[0]
            else:
                self.region_dropdown.options = [(name, code) for name, code in self.vertex_regions.items()]
                self.region_dropdown.value = "us-central1"
            conditional_widgets = [self.region_dropdown, self.project_textbox]
            
        else:
            conditional_widgets = [self.api_key_textbox]
            
        # elif provider == ModelProvider.Google:
        #     # Show region and project for Google
        #     self.region_dropdown.options = [(name, code) for name, code in self.google_regions.items()]
        #     self.region_dropdown.value = "us-central1"
        #     conditional_widgets = [self.region_dropdown, self.project_textbox]
            
        # Update conditional container
        self.conditional_container.children = conditional_widgets
        
        # Update output
        self.update_output()
        
    def on_model_change(self, change):
        """Handle model selection change."""
        provider = self.provider_dropdown.value
        model_code = change['new']
        
        # Update regions for AnthropicVertex models
        if provider == ModelProvider.AnthropicVertex:
            self.update_regions_for_model(model_code)
            
        self.update_output()
    
    def update_regions_for_model(self, model_code: str):
        """Update available regions based on selected Vertex model."""
        if model_code in self.vertex_model_regions:
            regions = self.vertex_model_regions[model_code]
            self.region_dropdown.options = [(name, code) for name, code in regions.items()]
            self.region_dropdown.value = list(regions.values())[0]  # Select first available region
        else:
            # Fallback to general vertex regions
            self.region_dropdown.options = [(name, code) for name, code in self.vertex_regions.items()]
            self.region_dropdown.value = "us-central1"
        self.update_output()
        
    def on_temperature_toggle(self, change):
        """Handle temperature checkbox toggle."""
        self.temperature_slider.disabled = change is None or not change['new']
        self.update_output()
        

    def update_output(self, change=None):
        """Update the output display with current selections."""
        with self.output:
            self.output.clear_output()
            
            provider = self.provider_dropdown.value
            model_code = self.model_dropdown.value if self.model_dropdown.options else ""
            model_name = self.get_model_display_name()
            
            print("Configuration Code:")
            print("=" * 50)
            
            # Generate the code snippet
            print("from jupyter_ai_agent_magics import config")
            print("config_manager = config.ConfigManager.instance")
            print()
            print(f"config_manager.model_provider = config.ModelProvider.{provider.name}")
            print(f'config_manager.model_name = "{model_code}"')
            
            if self.api_key_textbox.value:
                print(f'config_manager.api_key = "{self.api_key_textbox.value}"')
                
            if provider in [ModelProvider.AnthropicVertex, ModelProvider.Google]:
                if self.project_textbox.value:
                    print(f'config_manager.project_id = "{self.project_textbox.value}"')
                print(f'config_manager.location = "{self.region_dropdown.value}"')
                
            if self.use_temperature_checkbox.value:
                print(f'config_manager.temperature = {self.temperature_slider.value}')
            else:
                print('# config_manager.temperature = None  # Will use default')
            
            print("\n" + "=" * 50)
    
    def _print_configuration_code(self, provider: ModelProvider, model_code: str):
        """Print the code needed to configure this setup manually."""
        
        print("from jupyter_ai_agent_magics import config")
        print("config_manager = config.ConfigManager.instance")
        print()
        print(f"config_manager.model_provider = config.ModelProvider.{provider.name}")
        print(f'config_manager.model_name = "{model_code}"')
        
        if self.api_key_textbox.value:
            print(f'config_manager.api_key = "{self.api_key_textbox.value}"')
            
        if provider in [ModelProvider.AnthropicVertex, ModelProvider.Google]:
            if self.project_textbox.value:
                print(f'config_manager.project_id = "{self.project_textbox.value}"')
            print(f'config_manager.location = "{self.region_dropdown.value}"')
            
        if self.use_temperature_checkbox.value:
            print(f'config_manager.temperature = {self.temperature_slider.value}')
        else:
            print('# config_manager.temperature = None  # Will use default')
    
    def get_model_display_name(self) -> str:
        """Get the display name for the selected model."""
        if not self.model_dropdown.options:
            return ""
            
        selected_code = self.model_dropdown.value
        provider = self.provider_dropdown.value
        models = self.provider_models[provider]
        
        for display_name, code in models.items():
            if code == selected_code:
                return display_name
        return selected_code
        """Get the display name for the selected model."""
        if not self.model_dropdown.options:
            return ""
            
        selected_code = self.model_dropdown.value
        provider = self.provider_dropdown.value
        models = self.provider_models[provider]
        
        for display_name, code in models.items():
            if code == selected_code:
                return display_name
        return selected_code
    
    def get_config_dict(self) -> Dict[str, any]:
        """Get current configuration as dictionary."""
        provider = self.provider_dropdown.value
        config = {
            'provider': provider.value,
            'model_code': self.model_dropdown.value if self.model_dropdown.options else "",
            'model_display_name': self.get_model_display_name(),
            'api_key': self.api_key_textbox.value or None,
            'temperature': self.temperature_slider.value if self.use_temperature_checkbox.value else None
        }
        
        if provider in [ModelProvider.AnthropicVertex, ModelProvider.Google]:
            config['region'] = self.region_dropdown.value
            config['project_id'] = self.project_textbox.value or None
            
        return config
    
    def load_from_config(self):
        """Load configuration from ConfigManager if available."""
        if not self.config_manager:
            return
            
        try:
            # Load current provider
            current_provider = self.config_manager.model_provider
            self.provider_dropdown.value = current_provider
            
            # Load settings for current provider
            settings = self.config_manager.get_settings_for_provider(current_provider)
            
            # Set model
            if settings.model_name and settings.model_name in self.provider_models[current_provider].values():
                self.model_dropdown.value = settings.model_name
            
            # Set API key
            if settings.api_key:
                self.api_key_textbox.value = settings.api_key
                
            # Set region/project for Vertex/Google
            if current_provider in [ModelProvider.AnthropicVertex]:
                if settings.location:
                    self.region_dropdown.value = settings.location
                if settings.project_id:
                    self.project_textbox.value = settings.project_id

            # Set temperature
            if settings.temperature is not None:
                self.use_temperature_checkbox.value = True
                self.temperature_slider.value = settings.temperature
                self.temperature_slider.disabled = False
            else:
                self.use_temperature_checkbox.value = False
                self.temperature_slider.disabled = True
                
        except Exception as e:
            import traceback
            print(f"Warning: Could not load from config: {e} {traceback.format_exc()}")
    
    def save_to_config(self, button=None):
        """Save current configuration to ConfigManager."""
        if not self.config_manager:
            with self.output:
                print("No ConfigManager provided - cannot save")
            return
            
        try:
            provider = self.provider_dropdown.value
            
            # Create settings object
            settings = ModelProviderSettings(
                model_name=self.model_dropdown.value,
                api_key=self.api_key_textbox.value or None,
                temperature=self.temperature_slider.value if self.use_temperature_checkbox.value else None
            )
            
            # Add provider-specific settings
            if provider == ModelProvider.AnthropicVertex:
                settings.location = self.region_dropdown.value
                settings.project_id = self.project_textbox.value or None
            
            # Save to config manager
            self.config_manager.model_provider = provider
            self.config_manager.save_settings_for_provider(provider, settings)
            
            with self.output:
                print("✅ Configuration saved successfully!")
                self.update_output()
                
        except Exception as e:
            with self.output:
                print(f"❌ Error saving configuration: {e}")
    
    def display(self):
        """Display the widget."""
        display(self.container)

# Convenience functions
def create_model_selector_with_config(config_manager=None):
    """Create model selector connected to ConfigManager."""
    return MultiProviderModelSelector(config_manager)

def demo_with_config():
    """Demo with mock ConfigManager."""
    # You would replace this with your actual ConfigManager.instance
    # config_manager = ConfigManager.instance
    config_manager = config.ConfigManager.instance  # Replace with actual config manager
    
    selector = create_model_selector_with_config(config_manager)
    selector.display()
    
    return selector

# Simple usage without config manager
def create_simple_provider_selector():
    """Create simple selector without ConfigManager integration."""
    selector = MultiProviderModelSelector()
    return selector

if __name__ == "__main__":
    # Demo usage
    print("Creating Multi-Provider Model Selector...")
    selector = demo_with_config()

Creating Multi-Provider Model Selector...


In [14]:
from jupyter_ai_agent_magics import magics
magics.load_ipython_extension(get_ipython())

registering magic, creating llm server: vertex_anthropic


In [17]:
%%agent
what is your name? what company created you?

cell: what is your name? what company created you?



HTML(value='')

VBox()